In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy.stats
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

## Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'chris'/'2024.06.15-rat-neurons'
output_path = rd.rootdir/'output'/'lenti-exp98'
cache_path = output_path/'data.gzip'

rd.plot.plot_well_metadata(base_path/'metadata.yaml')

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channels = ['FSC-A', 'SSC-A', 'mGL-A', 'mRuby2-A', 'mCherry-A']
    data = rd.flow.load_csv_with_metadata(base_path/'export', base_path/'metadata.yaml', columns=channels)
    
    data.to_parquet(rd.outfile(cache_path))
# Create dicts to specify colors/markers
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
metadata['TS'] = metadata['ts_kind']
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']
display(data)

### Extremely basic analysis

In [ ]:
data['short_condition'] = data.virus.map({"pKG3097": "base", "RC130": "OL", "RC131": "CL"})

In [ ]:
mGL_gate = data[data.virus == 'untransduced']['mGL-A'].quantile(0.99999) * 1.1
mRuby2_gate = data[data.virus == 'untransduced']['mRuby2-A'].quantile(0.99999) * 1.1
livedead_gate = 2000

for channel, gate in zip(['mGL-A', 'mRuby2-A'], [mGL_gate, mRuby2_gate]):
    sns.kdeplot(data[data[channel] > 0], x=channel, log_scale=True, hue='virus')
    plt.axvline(gate, color='k', ls=':')
    plt.show()

In [ ]:
gated = data[(data['mGL-A'] > mGL_gate) & (data['mRuby2-A'] > mRuby2_gate)].copy()

In [ ]:
n_quantiles = 20
transfection_bin = gated.groupby(['virus', 'moi', 'short_condition']).apply(lambda df: pd.qcut(df['mGL-A'], n_quantiles, labels=np.linspace(0,1,n_quantiles, endpoint=False)), include_groups=False).reset_index().set_index('level_3')['mGL-A']
gated['transfection_bin'] = transfection_bin

In [ ]:
def compute_bin_gmeans(df):
    # Calculate the gmean by bin
    return df.loc[:, ('mGL-A', 'mRuby2-A')].apply(scipy.stats.gmean)
bin_gmeans = (gated.groupby(['short_condition', 'moi', 'transfection_bin'], observed=True)
      .apply(compute_bin_gmeans, include_groups=False)
      .rename(columns={"mGL-A": "mGL-bin-gmean", "mRuby2-A": "mRuby2-bin-gmean"}).reset_index())
gated_with_bins = pd.merge(gated, bin_gmeans, on=['short_condition', 'moi', 'transfection_bin'], validate="many_to_one")

In [ ]:
g = sns.FacetGrid(data=bin_gmeans, col='short_condition', col_order=['base', 'OL', 'CL'], hue='moi', palette='viridis', margin_titles=True)
g.map(sns.scatterplot, 'mGL-bin-gmean', 'mRuby2-bin-gmean')
for ax in g.axes.flatten():
    ax.set_xscale('log')
    ax.set_yscale('log')
plt.savefig(rd.outfile(output_path/'raw_gmeans_log.svg'), bbox_inches='tight')
plt.show()